In [30]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage, HumanMessage

In [31]:
load_dotenv()  # Load environment variables from .env file
generator_llm=AzureChatOpenAI(model="gpt-4o-mini")
evaluator_llm=AzureChatOpenAI(model="gpt-4o-mini")
optimizer_llm=AzureChatOpenAI(model="gpt-4o-mini")

In [32]:
import operator


class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iterations: int
    max_iterations: int
    
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [33]:
class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(description="Evaluation of the tweet")
    feedback: str = Field(description="Feedback for improvement if needed")

In [34]:
structured_evaluation_model = evaluator_llm.with_structured_output(TweetEvaluation)

In [35]:
def generate_tweet(state: TweetState) -> TweetState:
    messages=[
        SystemMessage(content="You are a creative Twitter content generator."),
        HumanMessage(content=f"""
                     Generate a tweet about the topic: {state['topic']}
                     Rules:
                        - Keep it under 280 characters.
                        - Make it engaging and relevant.
                        - Do not use question answer format.
                        - Use simple, day-to-day language.
                     """)
    ]
    response = generator_llm.invoke(messages).content
    return {'tweet': response, 'tweet_history': [response]}

def evaluate_tweet(state: TweetState):
    messages=[
        SystemMessage(content="You are a Twitter content evaluator."),
        HumanMessage(content=f"""
                     Evaluate the following tweet: {state['tweet']} based on the topic: {state['topic']}
                     Criteria:
                        - Relevance to the topic.
                        - Engagement potential.
                        - Clarity and conciseness.
                     Provide feedback and rate as 'approved' or 'needs_improvement'.
                     """)
    ]
    response = structured_evaluation_model.invoke(messages)
    return {'evaluation': response.evaluation, 'feedback': response.feedback, 'feedback_history':[response.feedback]}

def optimize_tweet(state: TweetState):
    messages=[
        SystemMessage(content="You are a Twitter content optimizer."),
        HumanMessage(content=f"""
                     Optimize the following tweet based on feedback: {state['tweet']}
                     Topic of the tweet: {state['topic']}
                     Feedback: {state['feedback']}
                     Rules:
                        - Address the feedback points.
                        - Maintain engagement and clarity.
                        - Keep it under 280 characters.
                     """)
    ]
    response = optimizer_llm.invoke(messages).content
    iterations = state['iterations'] + 1
    return {'tweet': response, 'iterations': iterations, 'tweet_history': [response]}

def route_evaluation(state: TweetState) -> str:
    if state['evaluation'] == 'approved' or state['iterations'] >= state['max_iterations']:
        return 'approved'
    else:
        return 'needs_improvement'

In [36]:
graph=StateGraph(TweetState)
graph.add_node('generate_tweet', generate_tweet)
graph.add_node('evaluate_tweet', evaluate_tweet)
graph.add_node('optimize_tweet', optimize_tweet)

graph.add_edge(START, 'generate_tweet')
graph.add_edge('generate_tweet', 'evaluate_tweet')
graph.add_conditional_edges('evaluate_tweet', route_evaluation, {'approved': END, 'needs_improvement': 'optimize_tweet'})
graph.add_edge('optimize_tweet', 'evaluate_tweet')

workflow=graph.compile()

In [42]:
initial_state={
    "topic": "adwqjhdda sdq",
    "iterations": 1,
    "max_iterations": 3
}
final_state=workflow.invoke(initial_state)
final_state

{'topic': 'adwqjhdda sdq',
 'tweet': "Life can feel as jumbled as 'adwqjhdda sdq'—a chaotic mess that mirrors our challenges! Just like untangling letters, every challenge is a puzzle to solve. Embrace the confusion and celebrate each little victory along the way! 🌟✨ #LifeJourney #StayPositive",
 'evaluation': 'needs_improvement',
 'feedback': "The tweet uses a jumbled string ('adwqjhdda sdq') as a metaphor for life challenges, which could confuse readers about the relevance of the topic. To improve, consider clarifying the connection between the jumbled letters and the message you want to convey about life's challenges. Additionally, while the sentiment is positive, making the tweet more concise could enhance engagement.",
 'iterations': 3,
 'max_iterations': 3,
 'tweet_history': ['Sometimes life feels like a jumble of letters just waiting to make sense! Embrace the chaos, find clarity in the confusion, and remember that every puzzle has a solution. Keep pushing through, and don’t for